In [0]:
import pandas as pd
import numpy as np
import keras
# from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import f1_score
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import pickle
import time
import tensorflow as tf

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [0]:
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return  1 - K.mean(f1)

In [0]:
start_time = time.time()
y = pd.read_csv("/content/gdrive/My Drive/data/y_bush_vs_others.csv", header= None)
y_bush = y.values.ravel()
y = pd.read_csv("/content/gdrive/My Drive/data/y_williams_vs_others.csv", header= None)
y_williams = y.values.ravel()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.021111011505126953 seconds ---


In [0]:
start_time = time.time()
X = pd.read_csv("/content/gdrive/My Drive/data/X.csv", sep=' ', header= None, dtype= float)
X = X.values
print("--- %s seconds ---" % (time.time() - start_time))

--- 24.795668601989746 seconds ---


In [0]:
X_train, X_test, y_bush_train, y_bush_test = train_test_split(X,y_bush, test_size=1./3, random_state = 6926, shuffle = True, stratify = y_bush)

In [0]:
X_train.shape

(8822, 4096)

In [0]:
X_test.shape

(4411, 4096)

In [0]:
X.shape

(13233, 4096)

In [0]:
X_train = X_train.reshape(8822,64,64,1)

In [0]:
X_test = X_test.reshape(4411,64,64,1)

In [0]:
y_bush_train = y_bush_train.reshape(8822,1)
y_bush_test = y_bush_test.reshape(4411,1)
# y_bush_train_c = 
# 

In [0]:
y_bush_train.shape

(8822, 1)

In [0]:
X_train.shape

(8822, 64, 64, 1)

In [0]:
X_test.shape

(4411, 64, 64, 1)

In [0]:
bush_model = Sequential()

In [0]:
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu',input_shape=(64,64,1)))
bush_model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'))
bush_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

bush_model.add(Flatten())
bush_model.add(Dense(1000,activation='relu'))
bush_model.add(Dense(1,activation='sigmoid'))

In [0]:
# bush_model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.05), metrics=[f1])
# bush_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=0.01),metrics=[f1])
# bush_model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=[f1])
bush_model.compile(loss=f1_loss, optimizer='adam', metrics=[f1])

In [0]:
csv_logger = keras.callbacks.CSVLogger('/content/gdrive/My Drive/data/bush_training.log')
bush_model.fit(X_train, y_bush_train, batch_size = 500, epochs = 100, verbose =1 , validation_data = (X_test, y_bush_test),callbacks=[csv_logger])
# bush_model.fit(X_train, y_bush_train_c, batch_size = 500, epochs = 10, verbose =1 , validation_data = (X_test, y_bush_test_c))

Train on 8822 samples, validate on 4411 samples
Epoch 1/100
8822/8822 [==============================] - 8s 932us/step - loss: 0.9236 - f1: 0.0737 - val_loss: 0.9229 - val_f1: 0.0771
Epoch 2/100
8822/8822 [==============================] - 4s 425us/step - loss: 0.9232 - f1: 0.0767 - val_loss: 0.9226 - val_f1: 0.0771
Epoch 3/100
8822/8822 [==============================] - 4s 425us/step - loss: 0.9102 - f1: 0.0979 - val_loss: 0.8754 - val_f1: 0.1355
Epoch 4/100
8822/8822 [==============================] - 4s 424us/step - loss: 0.8553 - f1: 0.1525 - val_loss: 0.8418 - val_f1: 0.1630
Epoch 5/100
8822/8822 [==============================] - 4s 425us/step - loss: 0.8265 - f1: 0.1841 - val_loss: 0.8487 - val_f1: 0.1539
Epoch 6/100
8822/8822 [==============================] - 4s 427us/step - loss: 0.7828 - f1: 0.2235 - val_loss: 0.7427 - val_f1: 0.2558
Epoch 7/100
8822/8822 [==============================] - 4s 426us/step - loss: 0.6753 - f1: 0.3333 - val_loss: 0.6537 - val_f1: 0.3500
Epoch 8

In [0]:
bush_model.save("/content/gdrive/My Drive/data/bush.model")